In [1]:
%pip install numpy pyvista simpleitk monai nibabel fire mlflow

# %% Imports
import os
import json
import torch
import numpy as np
import SimpleITK as sitk
import nibabel as nib
import matplotlib.pyplot as plt

from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config

print_config()

# %% [markdown]
# # Create Updated JSON Datalist

# Define dataset splits (update file paths)
sim_datalist = {
    "testing": [
        {"image": "imagesTr/Normal_5.nii.gz", "label": "labelsTr/Normal_5.nii.gz"},
        {"image": "imagesTr/Diseased_5.nii.gz", "label": "labelsTr/Diseased_5.nii.gz"},
        {"image": "imagesTr/Normal_10.nii.gz", "label": "labelsTr/Normal_10.nii.gz"},
        {"image": "imagesTr/Diseased_10.nii.gz", "label": "labelsTr/Diseased_10.nii.gz"},
        {"image": "imagesTr/Normal_15.nii.gz", "label": "labelsTr/Normal_15.nii.gz"},
        {"image": "imagesTr/Diseased_15.nii.gz", "label": "labelsTr/Diseased_15.nii.gz"},
        {"image": "imagesTr/Normal_20.nii.gz", "label": "labelsTr/Normal_20.nii.gz"},
        {"image": "imagesTr/Diseased_20.nii.gz", "label": "labelsTr/Diseased_20.nii.gz"}
    ],
    "training": [
        {"fold": 0, "image": "imagesTr/Normal_1.nii.gz", "label": "labelsTr/Normal_1.nii.gz"},
        {"fold": 0, "image": "imagesTr/Normal_2.nii.gz", "label": "labelsTr/Normal_2.nii.gz"},
        {"fold": 0, "image": "imagesTr/Normal_3.nii.gz", "label": "labelsTr/Normal_3.nii.gz"},
        {"fold": 0, "image": "imagesTr/Normal_4.nii.gz", "label": "labelsTr/Normal_4.nii.gz"},
        {"fold": 1, "image": "imagesTr/Normal_6.nii.gz", "label": "labelsTr/Normal_6.nii.gz"},
        {"fold": 1, "image": "imagesTr/Normal_7.nii.gz", "label": "labelsTr/Normal_7.nii.gz"},
        {"fold": 1, "image": "imagesTr/Normal_8.nii.gz", "label": "labelsTr/Normal_8.nii.gz"},
        {"fold": 1, "image": "imagesTr/Normal_9.nii.gz", "label": "labelsTr/Normal_9.nii.gz"},
        {"fold": 2, "image": "imagesTr/Normal_11.nii.gz", "label": "labelsTr/Normal_11.nii.gz"},
        {"fold": 2, "image": "imagesTr/Normal_12.nii.gz", "label": "labelsTr/Normal_12.nii.gz"},
        {"fold": 2, "image": "imagesTr/Normal_13.nii.gz", "label": "labelsTr/Normal_13.nii.gz"},
        {"fold": 2, "image": "imagesTr/Normal_14.nii.gz", "label": "labelsTr/Normal_14.nii.gz"},    
        {"fold": 3, "image": "imagesTr/Normal_16.nii.gz", "label": "labelsTr/Normal_16.nii.gz"},
        {"fold": 3, "image": "imagesTr/Normal_17.nii.gz", "label": "labelsTr/Normal_17.nii.gz"},
        {"fold": 3, "image": "imagesTr/Normal_18.nii.gz", "label": "labelsTr/Normal_18.nii.gz"},
        {"fold": 3, "image": "imagesTr/Normal_19.nii.gz", "label": "labelsTr/Normal_19.nii.gz"},
        {"fold": 0, "image": "imagesTr/Diseased_1.nii.gz", "label": "labelsTr/Diseased_1.nii.gz"},
        {"fold": 0, "image": "imagesTr/Diseased_2.nii.gz", "label": "labelsTr/Diseased_2.nii.gz"},
        {"fold": 0, "image": "imagesTr/Diseased_3.nii.gz", "label": "labelsTr/Diseased_3.nii.gz"},
        {"fold": 0, "image": "imagesTr/Diseased_4.nii.gz", "label": "labelsTr/Diseased_4.nii.gz"},
        {"fold": 1, "image": "imagesTr/Diseased_6.nii.gz", "label": "labelsTr/Diseased_6.nii.gz"},
        {"fold": 1, "image": "imagesTr/Diseased_7.nii.gz", "label": "labelsTr/Diseased_7.nii.gz"},
        {"fold": 1, "image": "imagesTr/Diseased_8.nii.gz", "label": "labelsTr/Diseased_8.nii.gz"},
        {"fold": 1, "image": "imagesTr/Diseased_9.nii.gz", "label": "labelsTr/Diseased_9.nii.gz"},
        {"fold": 2, "image": "imagesTr/Diseased_11.nii.gz", "label": "labelsTr/Diseased_11.nii.gz"},
        {"fold": 2, "image": "imagesTr/Diseased_12.nii.gz", "label": "labelsTr/Diseased_12.nii.gz"},
        {"fold": 2, "image": "imagesTr/Diseased_13.nii.gz", "label": "labelsTr/Diseased_13.nii.gz"},
        {"fold": 2, "image": "imagesTr/Diseased_14.nii.gz", "label": "labelsTr/Diseased_14.nii.gz"},
        {"fold": 3, "image": "imagesTr/Diseased_16.nii.gz", "label": "labelsTr/Diseased_16.nii.gz"},
        {"fold": 3, "image": "imagesTr/Diseased_17.nii.gz", "label": "labelsTr/Diseased_17.nii.gz"},
        {"fold": 3, "image": "imagesTr/Diseased_18.nii.gz", "label": "labelsTr/Diseased_18.nii.gz"},
        {"fold": 3, "image": "imagesTr/Diseased_19.nii.gz", "label": "labelsTr/Diseased_19.nii.gz"}
    ]
}

# Save datalist JSON
datalist_file = os.path.join("./2ndauto3dseg_dir", "sim_datalist.json")
with open(datalist_file, "w") as f:
    json.dump(sim_datalist, f)

print(f"✅ Datalist saved at: {datalist_file}")

work_dir = "./2ndauto3dseg_dir"
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

dataroot_dir = os.path.join(work_dir, "sim_dataroot")
#dataroot_dir = work_dir
if not os.path.isdir(dataroot_dir):
    os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "sim_datalist.json")
with open(datalist_file, "w") as f:
    json.dump(sim_datalist, f)


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


/home/jovyan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2025-04-01 16:29:18.217407: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-01 16:29:19.197044: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-01 16:29:19.197105: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT fa

MONAI version: 1.4.0
Numpy version: 1.26.4
Pytorch version: 2.3.1+cu118
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: /home/<username>/.local/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.4.3
Nibabel version: 5.3.2
scikit-image version: 0.25.1
scipy version: 1.15.2
Pillow version: 11.1.0
Tensorboard version: 2.14.1
gdown version: 5.2.0
TorchVision version: 0.18.1+cu118
tqdm version: 4.67.1
lmdb version: 1.6.2
psutil version: 7.0.0
pandas version: 2.2.3
einops version: 0.8.1
transformers version: 4.40.2
mlflow version: 2.21.0
pynrrd version: 1.1.3
clearml version: 1.18.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies

✅ Datalist saved at: ./2ndauto3dseg_dir/sim_datalist.json


In [2]:
# Define AutoRunner
input={
        "modality": "CT",
        "datalist": datalist_file,
        "dataroot": dataroot_dir,
    }

output_data_dir = "./2ndauto3dseg_dir/sim_dataroot"

runner = AutoRunner(
    input=input,
    work_dir="./2ndauto3dseg_dir",
    datalist_filename=datalist_file,  # Updated JSON with relative paths
    dataroot=output_data_dir,  # Directory containing converted .nii.gz files
    #analyze_params={"image_key":"image","label_key":"label"},
    algos=['dints']
)

# %% [markdown]
# # Configure Training Parameters

max_epochs = 100
train_param = {
    "num_epochs_per_validation": 5,
    "num_images_per_batch": 1,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 2,
}

runner.set_training_params(train_param)
runner.set_num_fold(num_fold=4)

# %% [markdown]
# # Run AutoRunner

print(torch.cuda.device_count())
#if torch.cuda.is_available():
#    # CUDA
#    gpu = torch.device('cuda:0')
#else:
#    # MacBook with >M1 chip
#    gpu = torch.device('mps')


#import torch
#print(torch.version.cuda)
#print(torch.backends.cudnn.version())
#CUDA_VISIBLE_DEVICES = [0,1]


#%env CUDA_VISIBLE_DEVICES=2
#runner.device_setting["CUDA_VISIBLE_DEVICES"] = ""  # Forces CPU mode
#runner.device_setting["n_devices"] = 1  # MONAI requires at least 1 device (CPU counts)

import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "1, 2, 3, 4"
print(torch.cuda.device_count())
print(f"Selected algorithms: {runner.algos}")

runner.run()

2025-04-01 16:29:39,664 - INFO - AutoRunner using work directory /home/jovyan/2ndauto3dseg_dir
2025-04-01 16:29:39,858 - INFO - Found num_fold 4 based on the input datalist /home/jovyan/2ndauto3dseg_dir/sim_datalist.json.
2025-04-01 16:29:39,860 - INFO - Setting num_fold 4 based on the input datalist /home/jovyan/2ndauto3dseg_dir/sim_datalist.json.
2025-04-01 16:29:39,866 - INFO - Using user defined command running prefix , will override other settings
8
8
Selected algorithms: ['dints']
2025-04-01 16:29:39,869 - INFO - Skipping data analysis...
2025-04-01 16:29:39,870 - INFO - Skipping algorithm generation...
2025-04-01 16:29:39,924 - INFO - Skipping already trained algos ['dints_0', 'dints_1'].Set option train=True to always retrain all algos.
2025-04-01 16:29:39,945 - INFO - The keys num_warmup_epochs cannot be found in the /home/jovyan/2ndauto3dseg_dir/dints_2/configs/hyper_parameters.yaml for training. Skipped overriding key num_warmup_epochs.
2025-04-01 16:29:39,950 - INFO - ['tor

W0401 16:29:41.857000 140538417358656 torch/distributed/run.py:757] 
W0401 16:29:41.857000 140538417358656 torch/distributed/run.py:757] *****************************************
W0401 16:29:41.857000 140538417358656 torch/distributed/run.py:757] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0401 16:29:41.857000 140538417358656 torch/distributed/run.py:757] *****************************************
/home/jovyan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/home/jovyan/.local/lib/python3.10/site-packages/matplotli

dints_2 - validation at original spacing/resolution


2025-04-01 17:42:32,070 - WARNING - dints_2 - training: finished


2025-04-01 17:42:51,086 - INFO - The keys num_warmup_epochs cannot be found in the /home/jovyan/2ndauto3dseg_dir/dints_3/configs/hyper_parameters.yaml for training. Skipped overriding key num_warmup_epochs.
2025-04-01 17:42:51,103 - INFO - ['torchrun', '--nnodes', '1', '--nproc_per_node', '8', '/home/jovyan/2ndauto3dseg_dir/dints_3/scripts/train.py', 'run', "--config_file='/home/jovyan/2ndauto3dseg_dir/dints_3/configs/hyper_parameters.yaml,/home/jovyan/2ndauto3dseg_dir/dints_3/configs/hyper_parameters_search.yaml,/home/jovyan/2ndauto3dseg_dir/dints_3/configs/network.yaml,/home/jovyan/2ndauto3dseg_dir/dints_3/configs/network_search.yaml,/home/jovyan/2ndauto3dseg_dir/dints_3/configs/transforms_infer.yaml,/home/jovyan/2ndauto3dseg_dir/dints_3/configs/transforms_train.yaml,/home/jovyan/2ndauto3dseg_dir/dints_3/configs/transforms_validate.yaml'", '--training#num_epochs_per_validation=5', '--training#num_images_per_batch=1', '--training#num_epochs=100']


...

2025-04-01 18:51:13,833 - INFO - The output_dir is not specified. /home/jovyan/2ndauto3dseg_dir/ensemble_output will be used to save ensemble predictions.
2025-04-01 18:51:13,956 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-04-01 18:51:13,956 - INFO - dints_0
2025-04-01 18:51:13,956 - INFO - dints_1
2025-04-01 18:51:13,956 - INFO - dints_2
2025-04-01 18:51:13,957 - INFO - dints_3
2025-04-01 18:51:13,957 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/jovyan/2ndauto3dseg_dir/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/1 [00:00<?, ?it/s]monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
monai.transforms.cr

2025-04-01 18:51:25.697472 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:51:25.697526 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:51:25.698279 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:51:25.699732 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:51:25.700198 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:51:25.705454 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:51:25.707659 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:51:25.708887 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:53:58.168757 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:54:12.094696 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:54:44.607075 - Length of input patch is recommended to be a multiple of 32.
2025-04-01

Ensembling (rank 0)...: 100%|██████████| 1/1 [14:16<00:00, 856.86s/it]


2025-04-01 19:07:10,774 - INFO - Auto3Dseg pipeline is completed successfully.
